In [ ]:
import pathlib
import pandas as pd
import numpy as np
import jupedsim as jps
import sqlite3
from shapely import Polygon
import plotly.graph_objects as go
import pedpy
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
%matplotlib inline

![](lane_formation_claudio.png)

In [ ]:
length = 38
width = 3
area = [[0, 0], [length, 0], [length, width], [0, width]]
exit_polygon_left = [(0, 0), (1, 0), (1, width), (0, width)]
exit_polygon_right = [
    (length - 1, 0),
    (length, 0),
    (length, width),
    (length - 1, width),
]
distribution_polygon_left = Polygon([[0, 0], [12, 0], [12, width], [0, width]])
distribution_polygon_right = Polygon(
    [[length - 12, 0], [length, 0], [length, width], [26, width]]
)
measurement_area = pedpy.MeasurementArea([[14, 0], [24, 0], [24, 3], [14, 3]])
measurement_line = pedpy.MeasurementLine([[14, 0], [14, width]])
walkable_area = pedpy.WalkableArea(area)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.set_aspect("equal")
pedpy.plot_measurement_setup(
    walkable_area=walkable_area,
    measurement_areas=[measurement_area],
    measurement_lines=[measurement_line],
    ml_color="red",
    ml_width=2,
    axes=ax,
)
for id, polygon in enumerate(
    [distribution_polygon_left, distribution_polygon_right]
):
    x, y = polygon.exterior.xy
    plt.fill(x, y, alpha=0.1, color="gray")
    centroid = polygon.centroid
    plt.text(
        centroid.x,
        centroid.y,
        f"Start {id+1}",
        ha="center",
        va="center",
        fontsize=10,
    )

In [ ]:
simulations = {}
COLUMNS = 9
number_agents = [
    (6*COLUMNS, 0*COLUMNS),
    (5*COLUMNS, 1*COLUMNS),
    (4*COLUMNS, 2*COLUMNS),
    (3*COLUMNS, 3*COLUMNS)
]
for number in number_agents:
    trajectory_file = f"trajectories_number_agents_{number}.sqlite"
    simulation = jps.Simulation(
        dt=0.05,
        model=jps.VelocityModelParameters(a_ped=2.6, d_ped=0.1, d_wall=0.05),
        geometry=walkable_area.polygon,
        trajectory_writer=jps.SqliteTrajectoryWriter(
            output_file=pathlib.Path(trajectory_file),
        ),
    )
    simulations[number] = simulation

In [ ]:

for number, simulation in simulations.items():
    exits = [simulation.add_exit_stage(exit_polygon_left),simulation.add_exit_stage(exit_polygon_right)]
    journeys = [
        simulation.add_journey(jps.JourneyDescription([exit])) for exit in exits
    ]       
    
    agent_parameters = jps.VelocityModelAgentParameters()
    agent_parameters.time_gap = 1
    agent_parameters.v0 = 1.2
    agent_parameters.radius = 0.2
    # first group
    agent_parameters.journey_id = journeys[0]
    agent_parameters.orientation = (-1,0)
    agent_parameters.stage_id = exits[0]
    positions = jps.distribute_by_number(polygon=distribution_polygon_right,
            number_of_agents=number[1],
            distance_to_agents=0.4,
            distance_to_polygon=0.7,
            seed=45131502,
        )
    for new_pos in positions:  
        agent_parameters.position = new_pos    
        id = simulation.add_agent(agent_parameters)

    # second group
    agent_parameters.journey_id = journeys[1]
    agent_parameters.orientation = (1,0)
    agent_parameters.stage_id = exits[1]
    positions = jps.distribute_by_number(polygon=distribution_polygon_left,
            number_of_agents=number[0],
            distance_to_agents=0.4,
            distance_to_polygon=0.7,
            seed=45131502,
        )
    for new_pos in positions:  
        agent_parameters.position = new_pos    
        simulation.add_agent(agent_parameters)


In [ ]:
def print_header(scenario_name: str):
    line_length = 50
    header = f" SIMULATION - {scenario_name} "
    left_padding = (line_length - len(header)) // 2
    right_padding = line_length - len(header) - left_padding

    print("=" * line_length)
    print(" " * left_padding + header + " " * right_padding)
    print("=" * line_length)

In [ ]:
trajectory_files = {}
for number, simulation in simulations.items():
    print_header(f"number {number}")
    agents=simulation.agents()
    while simulation.agent_count() > 0 and simulation.iteration_count() < 3000:
        simulation.iterate()
        
    
    trajectory_file = f"trajectories_number_agents_{number}.sqlite"
    trajectory_files[number] = trajectory_file
    # can I get trajectory_file from the simulation object?
    print(
        f"> Simulation completed after {simulation.iteration_count()} iterations.\n"
        f"> Output File: {trajectory_file}\n"
    )

In [ ]:
import matplotlib.pyplot as plt
DUMMY_SPEED = -1000
def read_sqlite_file(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        walkable_area = (
            con.cursor().execute("select wkt from geometry").fetchone()[0]
        )
        return (
            pedpy.TrajectoryData(data=data, frame_rate=fps),
            pedpy.WalkableArea(walkable_area),
        )


def speed_to_color(speed, min_speed, max_speed):
    """Map a speed value to a color using a colormap."""
    normalized_speed = (speed - min_speed) / (max_speed - min_speed)
    r, g, b = plt.cm.jet_r(normalized_speed)[:3]
    return f"rgba({r*255:.0f}, {g*255:.0f}, {b*255:.0f}, 0.5)"

def get_line_color(disk_color):

    r, g, b, _ = [int(float(val)) for val in disk_color[5:-2].split(",")]
    brightness = (r * 299 + g * 587 + b * 114) / 1000
    return "black" if brightness > 127 else "white"  

def create_orientation_line(row, line_length=0.2, color='black'):
    end_x = row["x"] + line_length * row["ox"]
    end_y = row["y"] + line_length * row["oy"]

    orientation_line = go.layout.Shape(
        type="line",
        x0=row["x"],
        y0=row["y"],
        x1=end_x,
        y1=end_y,
        line=dict(color=color, width=3),
    )
    return orientation_line

def get_geometry_traces(area):
    geometry_traces = []
    x, y = area.exterior.xy
    geometry_traces.append(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
            showlegend=False,
            name="Exterior",
            hoverinfo="name",
        )
    )
    for inner in area.interiors:
        xi, yi = zip(*inner.coords[:])
        geometry_traces.append(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
                showlegend=False,
                name="Obstacle",
                hoverinfo="name",
            )
        )
    return geometry_traces

def get_colormap(frame_data, max_speed):
    """Utilize scatter plots with varying colors for each agent instead of individual shapes.

    This trace is only to incorporate a colorbar in the plot.
    """
    scatter_trace = go.Scatter(
        x=frame_data["x"],
        y=frame_data["y"],
        mode="markers",
        marker=dict(
            size=frame_data["radius"] * 2,
            color=frame_data["speed"],
            colorscale="Jet_r",
            colorbar=dict(title="Speed [m/s]"),
            cmin=0,
            cmax=max_speed,
        ),
        text=frame_data["speed"],
        showlegend=False,
        hoverinfo="none",
    )

    return [scatter_trace]

def get_shapes_for_frame(frame_data, min_speed, max_speed):
    def create_shape(row):
        hover_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            text=[f"ID: {row['id']}, Pos({row['x']:.2f},{row['y']:.2f})"],
            mode="markers",
            marker=dict(size=1, opacity=1),
            hoverinfo="text",
            showlegend=False,
        )
        if row["speed"] == DUMMY_SPEED:
            dummy_trace = go.Scatter(
                x=[row["x"]],
                y=[row["y"]],
                mode="markers",
                marker=dict(size=1, opacity=0),
                hoverinfo="none",
                showlegend=False,
            )
            return (
                go.layout.Shape(
                    type="circle",
                    xref="x",
                    yref="y",
                    x0=row["x"] - row["radius"],
                    y0=row["y"] - row["radius"],
                    x1=row["x"] + row["radius"],
                    y1=row["y"] + row["radius"],
                    line=dict(width=0),
                    fillcolor="rgba(255,255,255,0)",  # Transparent fill
                ),
                dummy_trace, create_orientation_line(row, color="rgba(255,255,255,0)")
            )
        color = speed_to_color(row["speed"], min_speed, max_speed)
        return (
            go.layout.Shape(
                type="circle",
                xref="x",
                yref="y",
                x0=row["x"] - row["radius"],
                y0=row["y"] - row["radius"],
                x1=row["x"] + row["radius"],
                y1=row["y"] + row["radius"],
                line_color=color,
                fillcolor=color,
            ),
            hover_trace, create_orientation_line(row,color=get_line_color(color))
        )

    results = frame_data.apply(create_shape, axis=1).tolist()
    shapes = [res[0] for res in results]
    hover_traces = [res[1] for res in results]
    arrows =  [res[2] for res in results]
    return shapes, hover_traces, arrows

def create_fig(
    initial_agent_count,
    initial_shapes,
    initial_arrows,
    initial_hover_trace,
    initial_scatter_trace,
    geometry_traces,
    frames,
    steps,
    area_bounds,
    width=800,
    height=800,
):
    """Creates a Plotly figure with animation capabilities.

    Returns:
        go.Figure: A Plotly figure with animation capabilities.
    """

    minx, miny, maxx, maxy = area_bounds
    title = f"<b>Ratio: {min(number)/sum(number):.3}, Number of Agents: {initial_agent_count}</b>"
    fig = go.Figure(
        data=geometry_traces
        + initial_scatter_trace
        #+ hover_traces
        + initial_hover_trace,
        frames=frames,
        layout=go.Layout(
            shapes=initial_shapes + initial_arrows, title=title, title_x=0.5
        ),
    )
    fig.update_layout(
        updatemenus=[_get_animation_controls()],
        sliders=[_get_slider_controls(steps)],
        autosize=False,
        width=width,
        height=height,
        xaxis=dict(range=[minx - 0.5, maxx + 0.5]),
        yaxis=dict(
            scaleanchor="x", scaleratio=1, range=[miny - 0.5, maxy + 0.5]
        ),
    )

    return fig


def _get_animation_controls():
    """Returns the animation control buttons for the figure."""
    return {
        "buttons": [
            {
                "args": [
                    None,
                    {
                        "frame": {"duration": 100, "redraw": True},
                        "fromcurrent": True,
                    },
                ],
                "label": "Play",
                "method": "animate",
            },
            {
                "args": [
                    [None],
                    {
                        "frame": {"duration": 0, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 0},
                    },
                ],
                "label": "Pause",
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top",
    }


def _get_slider_controls(steps):
    """Returns the slider controls for the figure."""
    return {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right",
        },
        "transition": {"duration": 100, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": steps,
    }

def animate(
    number, data: pedpy.TrajectoryData, area: pedpy.WalkableArea, *, every_nth_frame=50, width=800, height=800
):
    data_df = pedpy.compute_individual_speed(traj_data=data, frame_step=5)
    data_df = data_df.merge(data.data, on=["id", "frame"], how="left")
    data_df["radius"] = 0.2
    min_speed = data_df["speed"].min()
    max_speed = data_df["speed"].max()
    max_agents = data_df.groupby("frame").size().max()
    frames = []
    steps = []
    unique_frames = data_df["frame"].unique()
    selected_frames = unique_frames[::every_nth_frame]
    geometry_traces = get_geometry_traces(area.polygon)
    initial_frame_data = data_df[data_df["frame"] == data_df["frame"].min()]
    initial_agent_count = len(initial_frame_data)
    initial_shapes, initial_hover_trace, initial_arrows = get_shapes_for_frame(
        initial_frame_data, min_speed, max_speed
    )
    color_map_trace = get_colormap(
        initial_frame_data, max_speed
    )

    for frame_num in selected_frames[1:]:
        frame_data, agent_count = _get_processed_frame_data(
            data_df, frame_num, max_agents
        )
        shapes, hover_traces, arrows = get_shapes_for_frame(
            frame_data, min_speed, max_speed
        )
        frame = go.Frame(
            data=geometry_traces + hover_traces,
            name=str(frame_num),
            layout=go.Layout(
                shapes=shapes + arrows,
                title=f"<b>Ratio: {min(number)/sum(number):.3}, Number of Agents: {agent_count}</b>",
                title_x=0.5,
            ),
        )
        frames.append(frame)

        step = {
            "args": [
                [str(frame_num)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 500},
                },
            ],
            "label": str(frame_num),
            "method": "animate",
        }
        steps.append(step)

    return create_fig(
        initial_agent_count,
        initial_shapes,
        initial_arrows,
        initial_hover_trace,
        color_map_trace,
        geometry_traces,
        frames,
        steps,
        area.bounds,
        width=width,
        height=height,
    )


def _get_processed_frame_data(data_df, frame_num, max_agents):
    """Process frame data and ensure it matches the maximum agent count."""
    frame_data = data_df[data_df["frame"] == frame_num]
    agent_count = len(frame_data)
    dummy_agent_data = {"x": 0, "y": 0, "radius": 0, "speed": DUMMY_SPEED}
    while len(frame_data) < max_agents:
        dummy_df = pd.DataFrame([dummy_agent_data])
        frame_data = pd.concat([frame_data, dummy_df], ignore_index=True)
    return frame_data, agent_count

In [ ]:
agent_trajectories = {}
for number in number_agents:
    trajectory_file = trajectory_files[number]
    agent_trajectories[number], walkable_area = read_sqlite_file(trajectory_file)
    animate(number, agent_trajectories[number], walkable_area, every_nth_frame=5, width=1200, height=400).show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

individual = {}
density_voronoi = {}
passing_density = {}
for number in number_agents:
    individual[number] = pedpy.compute_individual_voronoi_polygons(   
        traj_data=agent_trajectories[number], walkable_area=walkable_area
    )
    density_voronoi[number], intersecting = pedpy.compute_voronoi_density(
        individual_voronoi_data=individual[number], measurement_area=measurement_area
    )
    frames_in_area, used_area = pedpy.compute_frame_range_in_area(
        traj_data=agent_trajectories[number], measurement_line=measurement_line, width=10.0
    )
    passing_density[number] = pedpy.compute_passing_density(
        density_per_frame=density_voronoi[number], frames=frames_in_area
    )

In [ ]:
fig0, ax0 = plt.subplots(nrows=1, ncols=1);
labels = []
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i, number in enumerate(number_agents):
    pedpy.plot_density(
        density=density_voronoi[number], axes=ax0, color=colors[i]
    )
    labels.append(f"Ratio: {min(number)/sum(number):.3f}")

ax0.legend(labels)
plt.close()
#-------
labels2 = []
fig1, axes = plt.subplots(nrows=1, ncols=len(number_agents))
for i, number in enumerate(number_agents):
    plt.sca(axes[i]) 
    pedpy.plot_density_distribution(
        density=passing_density[number],
        title=f"Ratio: {min(number)/sum(number):.2f}",
        axes=axes[i],
        facecolor=colors[i],
        edgecolor=colors[i]
    )
    labels2.append(f"Ratio: {min(number)/sum(number):.2f}")
    axes[i].set_ylim(0, 1.5);

plt.close()
#-------
fig0.canvas.draw()
fig1.canvas.draw()
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(10, 5))

gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1]) 

ax1_new = fig.add_subplot(gs[0])
ax2_new = fig.add_subplot(gs[1])

ax1_new.imshow(fig0.canvas.buffer_rgba(), aspect='auto')
ax2_new.imshow(fig1.canvas.buffer_rgba(), aspect='auto')
ax2_new.set_title("Individual density inside measurement area")
ax1_new.axis('off')
ax2_new.axis('off')

plt.tight_layout()
plt.show()